### Importing Required Libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator


In [2]:
tf.__version__

'2.10.0'

### PreProcessing the Data

In [11]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)
training_set=train_datagen.flow_from_directory('dog_and_cat_dataset/training_set',
                                               target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='binary')

Found 8005 images belonging to 2 classes.


In [10]:
test_datagen=ImageDataGenerator(rescale=1./255)
test_set=test_datagen.flow_from_directory('dog_and_cat_dataset/test_set',
                                           target_size=(64, 64),
                                           batch_size=32,
                                           class_mode='binary')

Found 2023 images belonging to 2 classes.


### Building the CNN 

#### Initializing the CNN

In [12]:
cnn=tf.keras.models.Sequential()

2025-04-19 20:08:31.732754: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-19 20:08:31.734230: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


#### Step 1 : Convolution

In [13]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

#### Step 2 : Pooling

In [17]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#### Adding a second convolutional layer 

In [18]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#### Step 3 : Flattening

In [19]:
cnn.add(tf.keras.layers.Flatten())

#### Step 4 : Full Connection

In [20]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

#### Step 5 : Output layer

In [21]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Training the CNN 

#### Compiling the CNN

In [23]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#### Training the CNN on the training set and evaluating it on the testing set

In [24]:
cnn.fit(x = training_set,
        validation_data=test_set,
        epochs=25)

Epoch 1/25
251/251 [==============================] - 95s 376ms/step - loss: 0.6676 - accuracy: 0.5958 - val_loss: 0.6286 - val_accuracy: 0.6693
Epoch 2/25
251/251 [==============================] - 87s 348ms/step - loss: 0.6267 - accuracy: 0.6575 - val_loss: 0.6027 - val_accuracy: 0.6826
Epoch 3/25
251/251 [==============================] - 84s 337ms/step - loss: 0.5785 - accuracy: 0.6954 - val_loss: 0.5910 - val_accuracy: 0.6797
Epoch 4/25
251/251 [==============================] - 91s 362ms/step - loss: 0.5428 - accuracy: 0.7257 - val_loss: 0.5219 - val_accuracy: 0.7370
Epoch 5/25
251/251 [==============================] - 92s 368ms/step - loss: 0.5209 - accuracy: 0.7425 - val_loss: 0.5180 - val_accuracy: 0.7435
Epoch 6/25
251/251 [==============================] - 89s 357ms/step - loss: 0.4987 - accuracy: 0.7578 - val_loss: 0.4935 - val_accuracy: 0.7622
Epoch 7/25
251/251 [==============================] - 90s 358ms/step - loss: 0.4857 - accuracy: 0.7631 - val_loss: 0.5143 - val_ac

### Making a single prediction 

In [31]:
import numpy as np
from keras.preprocessing import image
from tensorflow.keras.utils import load_img, img_to_array
test_image=load_img('dog_and_cat_dataset/single_prediction/dog_and_cat_02.jpg',target_size=(64, 64))
test_image=img_to_array(test_image)
test_image=np.expand_dims(test_image, axis=0)
result= cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction='dog'
else:
    prediction='cat'

1/1 [==============================] - 0s 18ms/step


In [32]:
print(prediction)

cat


## Saving the CNN Model

In [33]:
cnn.save('dog_and_cat_model.h5')

### Code to load the model

In [34]:
from tensorflow.keras.models import load_model
model=load_model('dog_and_cat_model.h5')